In [ ]:
pip install sentencepiece

   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/992.0 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/992.0 kB ? eta -:--:--
   -------------------- ----------------- 524.3/992.0 kB 541.6 kB/s eta 0:00:01
   -------------------- ----------------- 524.3/992.0 kB 541.6 kB/s eta 0:00:01
   ------------------------------ ------- 786.4/992.0 kB 599.2 kB/s eta 0:00:01
   ------------------------------ ------- 786.4/992.0 kB 599.2 kB/s eta 0:00:01
   -------------------------------------- 992.0/992.0 kB 614.9 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install python-docx

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "deepvk/kazRush-ru-kk"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

d:\Programs\MiniConda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ramazan\.cache\huggingface\hub\models--deepvk--kazRush-ru-kk. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/788M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(8000, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(8000, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=4096, bias=False)
              (wi_1): Linear(in_features=512, out_features=4096, bias=False)
              (wo): Li

In [2]:
from docx import Document
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [11]:
def extract_all_texts(doc):
    texts = []

    for para in doc.paragraphs:
        texts.append(("p", para))  # обычный параграф

    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for para in cell.paragraphs:
                    texts.append(("p", para))  # параграф в таблице

    return texts

In [12]:
def translate_many_texts(texts):
    translated = []
    for _, para in tqdm(texts, desc="Translating text blocks"):
        if para.text.strip():
            translated_text = translate(para.text.strip())
        else:
            translated_text = para.text
        translated.append(translated_text)
    return translated

In [13]:
def replace_text_preserving_style(texts, translations):
    for (_, para), new_text in zip(texts, translations):
        if not para.runs:
            para.add_run(new_text)
            continue

        first_run = para.runs[0]
        para.clear()
        new_run = para.add_run(new_text)

        # Переносим стиль
        new_run.bold = first_run.bold
        new_run.italic = first_run.italic
        new_run.underline = first_run.underline
        new_run.font.name = first_run.font.name
        if first_run.font.size:
            new_run.font.size = first_run.font.size


In [14]:
def full_translate_docx(input_path, output_path):
    doc = Document(input_path)

    all_texts = extract_all_texts(doc)
    translations = translate_many_texts(all_texts)
    replace_text_preserving_style(all_texts, translations)

    doc.save(output_path)
    print(f"Перевод завершён: {output_path}")


In [16]:
input_file = "docs/Черноморский_флот_и_работорговля.docx"
output_file = "output_kazakh.docx"

full_translate_docx(input_file, output_file)

Translating text blocks: 100%|██████████| 2/2 [00:05<00:00,  2.80s/it]

Перевод завершён: output_kazakh.docx


In [33]:
def translate(text):
    """Функция перевода текста"""
    if not text or not text.strip():
        return text
    
    # Токенизация
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Генерация перевода
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=512,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=2
        )
    
    # Декодирование
    translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated

def extract_all_texts(doc):
    """Извлекает ВСЕ тексты из документа включая пропущенные ранее"""
    texts = []
    
    # 1. Основные параграфы документа (включая пустые, но проверяем текст)
    for para in doc.paragraphs:
        # Берем ВСЕ параграфы, даже если они кажутся пустыми
        # Иногда в них есть только пробелы или скрытое форматирование
        texts.append(("paragraph", para))
    
    # 2. Текст в таблицах
    for table_idx, table in enumerate(doc.tables):
        for row_idx, row in enumerate(table.rows):
            for cell_idx, cell in enumerate(row.cells):
                for para in cell.paragraphs:
                    # Берем все параграфы в ячейках
                    texts.append(("table_cell", para, (table_idx, row_idx, cell_idx)))
    
    # 3. Заголовки и колонтитулы
    for section_idx, section in enumerate(doc.sections):
        # Верхний колонтитул
        if section.header:
            for para in section.header.paragraphs:
                texts.append(("header", para, section_idx))
        
        # Нижний колонтитул
        if section.footer:
            for para in section.footer.paragraphs:
                texts.append(("footer", para, section_idx))
    
    # 4. Дополнительная проверка на пропущенный контент
    try:
        # Проверяем, есть ли в документе элементы, которые мы могли пропустить
        from docx.document import Document as DocxDocument
        # Можно добавить дополнительные проверки здесь
        pass
    except:
        pass
    
    # Фильтруем действительно пустые элементы только после извлечения
    filtered_texts = []
    for item in texts:
        if len(item) == 2:  # старый формат (тип, параграф)
            text_type, para = item
            extra_info = None
        else:  # новый формат с дополнительной информацией
            text_type, para, extra_info = item
        
        # Проверяем, есть ли реальный текст для перевода
        if para.text.strip():
            if extra_info is not None:
                filtered_texts.append((text_type, para, extra_info))
            else:
                filtered_texts.append((text_type, para))
    
    print(f"Найдено {len(filtered_texts)} текстовых блоков для перевода")
    return filtered_texts

def translate_many_texts(texts):
    """Переводит все тексты с показом прогресса - улучшенная версия"""
    translated = []
    
    for item in tqdm(texts, desc="Переводим тексты"):
        # Обрабатываем разные форматы элементов
        if len(item) == 2:
            text_type, para = item
        else:
            text_type, para, extra_info = item
        
        original_text = para.text.strip()
        
        if original_text:
            # Разбиваем длинные тексты на предложения для лучшего качества перевода
            if len(original_text) > 400:
                # Улучшенное разбиение на предложения
                sentences = re.split(r'(?<=[.!?])\s+', original_text)
                translated_sentences = []
                
                for sentence in sentences:
                    sentence = sentence.strip()
                    if sentence:
                        translated_sentence = translate(sentence)
                        translated_sentences.append(translated_sentence)
                
                # Восстанавливаем структуру текста
                translated_text = ' '.join(translated_sentences)
                
                # Сохраняем окончания предложений
                if original_text.endswith('.') and not translated_text.endswith('.'):
                    translated_text += '.'
                elif original_text.endswith('!') and not translated_text.endswith('!'):
                    translated_text += '!'
                elif original_text.endswith('?') and not translated_text.endswith('?'):
                    translated_text += '?'
            else:
                translated_text = translate(original_text)
        else:
            translated_text = original_text
        
        translated.append(translated_text)
    
    return translated

def replace_text_preserving_style(texts, translations):
    """Заменяет текст, сохраняя ВСЕ стили и форматирование - улучшенная версия"""
    
    for item, new_text in zip(texts, translations):
        # Обрабатываем разные форматы элементов
        if len(item) == 2:
            text_type, para = item
        else:
            text_type, para, extra_info = item
        
        if not para.runs:
            # Если нет runs, просто добавляем текст
            if new_text.strip():  # добавляем только если есть текст
                para.add_run(new_text)
            continue
        
        # Сохраняем стили ВСЕХ runs (улучшенная версия)
        run_styles = []
        for run in para.runs:
            style_info = {
                'bold': run.bold,
                'italic': run.italic,
                'underline': run.underline,
                'font_name': run.font.name,
                'font_size': run.font.size,
                'font_color': None,
                'highlight_color': None,
                # Дополнительные стили
                'subscript': run.font.subscript,
                'superscript': run.font.superscript,
                'strike': run.font.strike,
                'all_caps': run.font.all_caps,
                'small_caps': run.font.small_caps,
            }
            
            # Безопасно сохраняем цвета
            try:
                if run.font.color.rgb:
                    style_info['font_color'] = run.font.color.rgb
            except:
                pass
            
            try:
                if hasattr(run.font, 'highlight_color') and run.font.highlight_color:
                    style_info['highlight_color'] = run.font.highlight_color
            except:
                pass
            
            run_styles.append(style_info)
        
        # Сохраняем форматирование параграфа
        para_formatting = {
            'alignment': para.alignment,
            'style': para.style,
        }
        
        # Сохраняем дополнительное форматирование параграфа
        try:
            pf = para.paragraph_format
            para_formatting.update({
                'left_indent': pf.left_indent,
                'right_indent': pf.right_indent,
                'first_line_indent': pf.first_line_indent,
                'space_before': pf.space_before,
                'space_after': pf.space_after,
                'line_spacing': pf.line_spacing,
            })
        except:
            pass
        
        # Очищаем параграф
        para.clear()
        
        # Добавляем переведенный текст с сохранением стиля
        if run_styles and new_text.strip():
            new_run = para.add_run(new_text)
            first_style = run_styles[0]
            
            # Применяем основные стили
            new_run.bold = first_style['bold']
            new_run.italic = first_style['italic']
            new_run.underline = first_style['underline']
            
            # Применяем стили шрифта
            if first_style['font_name']:
                new_run.font.name = first_style['font_name']
            if first_style['font_size']:
                new_run.font.size = first_style['font_size']
            if first_style['font_color']:
                try:
                    new_run.font.color.rgb = first_style['font_color']
                except:
                    pass
            
            # Применяем дополнительные стили
            try:
                new_run.font.subscript = first_style['subscript']
                new_run.font.superscript = first_style['superscript']
                new_run.font.strike = first_style['strike']
                new_run.font.all_caps = first_style['all_caps']
                new_run.font.small_caps = first_style['small_caps']
            except:
                pass
        
        # Восстанавливаем форматирование параграфа
        try:
            if para_formatting['alignment'] is not None:
                para.alignment = para_formatting['alignment']
            if para_formatting['style']:
                para.style = para_formatting['style']
            
            # Восстанавливаем дополнительное форматирование
            pf = para.paragraph_format
            for attr in ['left_indent', 'right_indent', 'first_line_indent', 
                        'space_before', 'space_after', 'line_spacing']:
                if para_formatting.get(attr) is not None:
                    try:
                        setattr(pf, attr, para_formatting[attr])
                    except:
                        pass
        except:
            pass

def full_translate_docx(input_path, output_path):
    """Полный перевод DOCX файла - улучшенная версия"""
    print(f"Загружаем документ: {input_path}")
    doc = Document(input_path)
    
    print("Извлекаем тексты...")
    all_texts = extract_all_texts(doc)
    
    if not all_texts:
        print("⚠️ Не найдено текстов для перевода!")
        return
    
    print("Переводим тексты...")
    translations = translate_many_texts(all_texts)
    
    print("Заменяем тексты с сохранением стилей...")
    replace_text_preserving_style(all_texts, translations)
    
    print("Сохраняем результат...")
    doc.save(output_path)
    print(f"✅ Перевод завершён: {output_path}")

def analyze_document_structure(input_path):
    """Анализирует структуру документа - улучшенная версия"""
    doc = Document(input_path)
    
    print(f"📄 Детальный анализ документа: {input_path}")
    print(f"   Всего параграфов: {len(doc.paragraphs)}")
    print(f"   Таблиц: {len(doc.tables)}")
    print(f"   Секций: {len(doc.sections)}")
    
    # Подсчет непустых параграфов
    non_empty_paras = [p for p in doc.paragraphs if p.text.strip()]
    print(f"   Непустых параграфов: {len(non_empty_paras)}")
    
    # Детальный анализ параграфов
    paras_with_runs = len([p for p in doc.paragraphs if p.runs])
    total_runs = sum(len(p.runs) for p in doc.paragraphs)
    print(f"   Параграфов с runs: {paras_with_runs}")
    print(f"   Всего runs: {total_runs}")
    
    # Подсчет ячеек в таблицах
    table_cells = 0
    table_cell_runs = 0
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for para in cell.paragraphs:
                    if para.text.strip():
                        table_cells += 1
                        table_cell_runs += len(para.runs)
    
    print(f"   Ячеек таблиц с текстом: {table_cells}")
    print(f"   Runs в таблицах: {table_cell_runs}")
    
    # Анализ заголовков и колонтитулов
    headers_footers = 0
    for section in doc.sections:
        if section.header:
            headers_footers += len([p for p in section.header.paragraphs if p.text.strip()])
        if section.footer:
            headers_footers += len([p for p in section.footer.paragraphs if p.text.strip()])
    
    print(f"   Заголовки/колонтитулы: {headers_footers}")
    
    total_text_elements = len(non_empty_paras) + table_cells + headers_footers
    print(f"   📊 Итого текстовых элементов: {total_text_elements}")
    
    return total_text_elements

# Функция для проверки качества извлечения
def debug_extraction(input_path, max_examples=5):
    """Показывает примеры извлеченного текста для отладки"""
    doc = Document(input_path)
    all_texts = extract_all_texts(doc)
    
    print(f"\n🔍 Примеры извлеченного текста (первые {max_examples}):")
    for i, item in enumerate(all_texts[:max_examples]):
        if len(item) == 2:
            text_type, para = item
            location = ""
        else:
            text_type, para, extra_info = item
            location = f" {extra_info}" if extra_info else ""
        
        text_preview = para.text[:100] + "..." if len(para.text) > 100 else para.text
        print(f"   {i+1}. [{text_type}{location}]: {repr(text_preview)}")

# Использование
if __name__ == "__main__":
    input_file = "docs/aaaaaa.docx"
    output_file = "output_kazakh.docx"
    
    # Сначала анализируем структуру
    total_texts = analyze_document_structure(input_file)
    print(f"\n🎯 Ожидается перевести {total_texts} текстовых блоков")
    
    # Показываем примеры извлечения (для отладки)
    debug_extraction(input_file)
    
    print(f"\n🚀 Начинаем перевод...")
    # Выполняем перевод
    full_translate_docx(input_file, output_file)

📄 Детальный анализ документа: docs/aaaaaa.docx
   Всего параграфов: 64
   Таблиц: 3
   Секций: 1
   Непустых параграфов: 34
   Параграфов с runs: 34
   Всего runs: 124
   Ячеек таблиц с текстом: 191
   Runs в таблицах: 12827
   Заголовки/колонтитулы: 0
   📊 Итого текстовых элементов: 225

🎯 Ожидается перевести 225 текстовых блоков
Найдено 225 текстовых блоков для перевода

🔍 Примеры извлеченного текста (первые 5):
   1. [paragraph]: 'Федеральное государственное бюджетное образовательное'
   2. [paragraph]: 'учреждение высшего профессионального образования'
   3. [paragraph]: 'Московский государственный университет имени М.В. Ломоносова'
   4. [paragraph]: 'Факультет журналистики'
   5. [paragraph]: 'РАБОЧАЯ ПРОГРАММА ДИСЦИПЛИНЫ'

🚀 Начинаем перевод...
Загружаем документ: docs/aaaaaa.docx
Извлекаем тексты...
Найдено 225 текстовых блоков для перевода
Переводим тексты...


Переводим тексты:  43%|████▎     | 96/225 [00:59<00:35,  3.67it/s]Both `max_new_tokens` (=127) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=127) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=127) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=127) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggi

Заменяем тексты с сохранением стилей...
Сохраняем результат...
✅ Перевод завершён: output_kazakh.docx
